Reference: https://www.kaggle.com/rahulrrrd/statistical-and-cluster-analysis/comments

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Data preparation

## Data overview

In [ ]:
products_df = pd.read_csv("/kaggle/input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv")
categories_df = pd.read_csv("/kaggle/input/summer-products-and-sales-in-ecommerce-wish/unique-categories.sorted-by-count.csv")

In [ ]:
products_df.head()

## Explore missing values

In [ ]:
def plot_missing_data(df):
    columns_with_null = df.columns[df.isna().sum() > 0]
    null_pct = (df[columns_with_null].isna().sum() / df.shape[0]).sort_values(ascending=False) * 100
    plt.figure(figsize=(8,6));
    sns.barplot(y = null_pct.index, x = null_pct, orient='h')
    plt.title('% Na values in dataframe by columns');

In [ ]:
plot_missing_data(products_df)

merchant_profile_picture contains the url to merchants profile picture where more than 80% of data is missing, so we can definitely drop it

In [ ]:
products_df.drop("merchant_profile_picture", axis=1, inplace=True)

In [ ]:
print("Unique values: ", products_df["urgency_text"].unique(), "\n")
print("Value counts:\n", products_df['urgency_text'].value_counts())

'Quantité limitée !' translates to 'Quantity Limited' and 'Réduction sur les achats en gros' means 'discount on wholesale purchases' rest are all nan, let's make them right

In [ ]:
products_df['urgency_text'] = products_df['urgency_text'].replace({
        'Quantité limitée !': 'QuantityLimited',
        'Réduction sur les achats en gros': 'WholesaleDiscount',
        np.nan: 'noText'
})
print(products_df["urgency_text"].value_counts())

In [ ]:
print("Unique values: ", products_df["has_urgency_banner"].unique(), "\n")
print("Value counts:\n", products_df["has_urgency_banner"].value_counts())

has_urgency_banner is a binary column which tells us whether the product has an urgency banner or not, so we can replace nan's with 0's to get rid of the nan's

In [ ]:
products_df['has_urgency_banner'] = products_df['has_urgency_banner'].replace(np.nan,0)
print(products_df["has_urgency_banner"].value_counts())

Now let's process the columns with ratings, all the rating count columns has same number of values missing i.e. 45 missing values but the rating_count column has no na values, lets check the rating_count where values are mising in other rating count columns

In [ ]:
rating_columns = ['rating_one_count','rating_two_count','rating_three_count','rating_four_count','rating_five_count']
products_df[rating_columns] = products_df[rating_columns].fillna(value=-1)
products_df.loc[products_df['rating_five_count'] == -1, 'rating_count'].value_counts()

all values in the rating_count column are 0 where there are na values in other rating count columns so lets fill 0 in place of the na values

In [ ]:
products_df[rating_columns] = products_df[rating_columns].replace(-1, 0)

Data in the remaining columns is categorical, so we can replace the nan values with a token 'Unknown'

In [ ]:
nan_cat_cols = ['origin_country','product_color','product_variation_size_id','merchant_name','merchant_info_subtitle']
products_df[nan_cat_cols] = products_df[nan_cat_cols].replace(np.nan, 'Unknown')

In [ ]:
products_df.isnull().sum()[products_df.isnull().sum() > 0]

## Drop useless features

In [ ]:
products_df.drop([
                  "product_picture",
                  "title",
                  "title_orig",
                  'merchant_title',
                  'merchant_name',
                  'merchant_info_subtitle',
                  "product_url",
                  "tags",
                  'rating_five_count',
                  'rating_four_count',
                  'rating_three_count',
                  'rating_two_count',
                  'rating_one_count',
                  "crawl_month"], axis=1, inplace=True)

# Test categorical encoders

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

def get_score(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    model = LinearRegression().fit(X_train, y_train)
    return mean_absolute_error(y_test, model.predict(X_test))

def test_on_targets(df):
    target = "units_sold"
    X_u = df.drop(target, axis=1)
    y_u = df[target]
    scaler = StandardScaler()
    X_u = pd.DataFrame(scaler.fit_transform(X_u), index=X_u.index, columns=X_u.columns)
    units_mae = get_score(X_u, y_u)

    target = "rating"
    X_r = df.drop(target, axis=1)
    y_r = df[target]
    scaler = StandardScaler()
    X_r = pd.DataFrame(scaler.fit_transform(X_r), index=X_r.index, columns=X_r.columns)
    rating_mae = get_score(X_r, y_r)
    return units_mae, rating_mae

## Ordinal Encoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders.count import CountEncoder
from category_encoders import CatBoostEncoder
from category_encoders.target_encoder import TargetEncoder

df = products_df.copy()
cat_cols = df.select_dtypes(include=["O"]).columns
oe = OrdinalEncoder()
df[cat_cols] = oe.fit_transform(df[cat_cols])
oe_units_mae, oe_rating_mae = test_on_targets(df)
oe_units_mae, oe_rating_mae

## Label Encoder

In [ ]:
df = products_df.copy()
le = LabelEncoder()
for cat_col in cat_cols:
    df[cat_col] = le.fit_transform(df[cat_col])
    
le_units_mae, le_rating_mae = test_on_targets(df)
le_units_mae, le_rating_mae

## One Hot Encoder

In [ ]:
df = products_df.copy()
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
oh_cols = pd.DataFrame(ohe.fit_transform(df[cat_cols]))
oh_cols.index = df.index
num_cols = df.drop(cat_cols, axis=1)
df = pd.concat([num_cols, oh_cols], axis=1)

ohe_units_mae, ohe_rating_mae = test_on_targets(df)
ohe_units_mae, ohe_rating_mae

## Count Encoder

In [ ]:
df = products_df.copy()
ce = CountEncoder()
df[cat_cols] = ce.fit_transform(df[cat_cols])
ce_units_mae, ce_rating_mae = test_on_targets(df)
ce_units_mae, ce_rating_mae

## Cat Boost Encoder

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

categorical_transformer = Pipeline(steps=[('cbe', CatBoostEncoder())])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_cols),
    ])

model = LinearRegression()
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', model)
                      ])

target = "units_sold"
df = products_df.copy()
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
pipe.fit(X_train, y_train)
cbe_units_mae = mean_absolute_error(y_test, pipe.predict(X_test))

target = "rating"
df = products_df.copy()
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
pipe.fit(X_train, y_train)
cbe_rating_mae = mean_absolute_error(y_test, pipe.predict(X_test))
cbe_units_mae, cbe_rating_mae

## Target Encoder

In [ ]:
categorical_transformer = Pipeline(steps=[('te', TargetEncoder())])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_cols),
    ])

model = LinearRegression()
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', model)
                      ])

target = "units_sold"
df = products_df.copy()
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
pipe.fit(X_train, y_train)
te_units_mae = mean_absolute_error(y_test, pipe.predict(X_test))

target = "rating"
df = products_df.copy()
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
pipe.fit(X_train, y_train)
te_rating_mae = mean_absolute_error(y_test, pipe.predict(X_test))
te_units_mae, te_rating_mae

In [ ]:
cat_encoders = pd.DataFrame([[ohe_units_mae, ohe_rating_mae], [ce_units_mae, ce_rating_mae], [te_units_mae, te_rating_mae], [cbe_units_mae, cbe_rating_mae], [oe_units_mae, oe_rating_mae], [le_units_mae, le_rating_mae]],
                            columns=["Units_sold", "Rating"], index=["One Hot Encoder", "Count Encoder", "Target Encoder", "Cat Boost Encoder", "Ordinal Encoder", "Label Encoder"])
cat_encoders.sort_values(by="Units_sold")

In [ ]:
cat_encoders.sort_values(by="Rating")

## Final df

In [ ]:
oe = OrdinalEncoder()
df = products_df.copy()
df[cat_cols] = oe.fit_transform(df[cat_cols])

# Explore correlation with target values

In [ ]:
import statsmodels.api as sm

def get_reg_summary(X, y):
    X_sm = sm.add_constant(X)
    est = sm.OLS(y, X_sm)
    est2 = est.fit()
    return est2.params

In [ ]:
target = "units_sold"
X = df.drop(target, axis=1)
y = df[target]
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)

## Negative correlations

The higher the coefficient, the lower target value will be

In [ ]:
coeffs = get_reg_summary(X, y).sort_values().drop("const")
sns.barplot(x=coeffs.head(10).values, y=coeffs.head(10).index)
plt.show()

## Positive correlations

The higher the coefficient, the greater target value will be

In [ ]:
sns.barplot(x=coeffs.tail(10).values, y=coeffs.tail(10).index)
plt.show()

In [ ]:
target = "rating"
X = df.drop(target, axis=1)
y = df[target]
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)

## Negative correlations

In [ ]:
coeffs = get_reg_summary(X, y).sort_values().drop("const")
sns.barplot(x=coeffs.head(10).values, y=coeffs.head(10).index)
plt.show()

## Positive correlations

In [ ]:
sns.barplot(x=coeffs.tail(10).values, y=coeffs.tail(10).index)
plt.show()